In [3]:
!pip install transformers
!pip install sentencepiece

In [11]:
pip install transformers==3.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.0/769.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 12.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 59.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [4]:
from google.colab import drive
#drive.mount('/content/drive')

In [18]:
from transformers import AutoModel, AutoTokenizer
import re
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


def generate(article_text,target_lang):
  input_ids = tokenizer(
      [WHITESPACE_HANDLER(article_text)],
      return_tensors="pt",
      padding="max_length",
      truncation=True,
      max_length=512
  )["input_ids"]
  output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
  )[0]
  return output_ids


path="news_data_10.csv"

df = pd.read_csv(path)
print(df.head())


n=len(df)


target_lang = "tamil" #give target language
given_id=[]
generated_summary=[]
for i in range(n):
  article_text=df.iloc[i]['article']
  given_id.append(df.iloc[i]['id'])
  output_ids=generate(article_text,target_lang)
  summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
  )
  generated_summary.append(summary)
  print(i)

k = pd.DataFrame({'id':given_id,'summary':generated_summary})

# saving the dataframe
k.to_csv('summXL_10.csv')



   id                                            article
0   1  நாலு ஆள் உயரம், முறுக்கு மீசை, கையில் வீச்சரிவ...
1   2  அமானுஷ்யமான சம்பவங்கள் நம்மை சுற்றி ஆங்காங்கே ...
2   3  காமன்வெல்த் போட்டி ஏற்பாட்டில் நடைபெற்ற முறைகே...
3   4  தென்அமெரிக்க நாடான பெருவில் காடுகள் மிகவும் பய...
4   5  கடந்த 18ம் தேதி சாயங்காலம்... அடைமழையை கிழித்த...
0
1
2
3
4
5
6
7
8
9


In [ ]:
import shutil
shutil.copy('summXL_10.csv', '/content/drive/My Drive/0Fyp/summ')